In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime,date,timedelta,timezone
import pytz
import json

# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import seaborn as sns

from tensorflow.keras.models import load_model
import joblib

from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest


In [2]:
# import functions_framework
# @functions_framework.http
# def forecast_asset_movement(request):


# Parameter

In [3]:
#https://stackoverflow.com/questions/61573102/calling-a-google-cloud-function-from-within-python
#https://medium.com/google-cloud/setup-and-invoke-cloud-functions-using-python-e801a8633096
#https://medium.com/google-cloud/gcp-cloud-functions-develop-it-the-right-way-82e633b07756

model_id='spy-ema1'
today='2023-05-26'

# if request.get_json():
#     print("JSON Post Date Info") # Post Method
#     request_json = request.get_json()
#     today=request_json['TODAY']
#     model_id=request_json['MODEL_ID']
# else:
#     print("Enviroment Variable Info")
#     today=os.environ.get('TODAY', '') 
#     #today=os.environ.get('TODAY', '2023-04-28')  
#     model_id=os.environ.get('MODEL_ID', 'spy-ema1')  
print("List parameter as belows")
print(f"today={today}")
print(f"model_id={model_id}")


loadModelMode='local'   # local,gcs

List parameter as belows
today=2023-05-26
model_id=spy-ema1


# BigQuery Setting

In [4]:
projectId='pongthorn'
dataset_id='FinAssetForecast'
table_id = f"{projectId}.{dataset_id}.fin_movement_forecast"
table_data_id = f"{projectId}.{dataset_id}.fin_data"
table_model_id= f"{projectId}.{dataset_id}.model_ts_metadata"

print(table_id)
print(table_data_id)
print(table_model_id)

# json_credential_file=r'C:\Windows\pongthorn-5decdc5124f5.json'
# credentials = service_account.Credentials.from_service_account_file(json_credential_file)
# client = bigquery.Client(project=projectId,credentials=credentials )
client = bigquery.Client(project=projectId )

def load_data_bq(sql:str):
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df


pongthorn.FinAssetForecast.fin_movement_forecast
pongthorn.FinAssetForecast.fin_data
pongthorn.FinAssetForecast.model_ts_metadata


# Load Model MetaData Configuration

In [5]:
sqlModelMt=f"""
SELECT * FROM `{table_model_id}`  where model_id='{model_id}'
"""
print(sqlModelMt)
dfModelMeta=load_data_bq(sqlModelMt)
if dfModelMeta.empty==False:
    modelMeta=dfModelMeta.iloc[0,:]
    asset_name=modelMeta['asset']
    prediction_name=modelMeta['prediction']
    model_path=modelMeta['gs_model_path']
    model_file=modelMeta['model_file']
    scaler_file=modelMeta['scaler_file']
    scalerPred_file=modelMeta['scaler_pred_file']
    print(modelMeta)
    
else: 
    raise Exception(f"Not found model id  {model_id}")



# print(f"{today} - {asset_name} -{prediction_name}")
# print("==================================================")


SELECT * FROM `pongthorn.FinAssetForecast.model_ts_metadata`  where model_id='spy-ema1'

model_id                                                     spy-ema1
asset                                                             SPY
prediction                                                       EMA1
gs_model_path              gs://demo-ts-forecast-pongthorn/model-ema1
model_file                   EMA1_60To10_SPY_E150S20-Y2015-2023_ma.h5
scaler_file              scaler_EMA1_60To10_SPY_E150S20-Y2015-2023.gz
scaler_pred_file    scaler_pred_EMA1_60To10_SPY_E150S20-Y2015-2023.gz
Name: 0, dtype: object


# Load model and scaler

In [6]:
if loadModelMode=='local':
 objectPaht="model/model-ema1"
else:
 objectPaht=model_path  

model_path=f"{objectPaht}/{model_file}"
scale_input_path=f"{objectPaht}/{scaler_file}"
scale_output_path=f"{objectPaht}/{scalerPred_file}"

print(f"load model from {loadModelMode}")   
print(model_path)
print(scale_input_path)
print(scale_output_path)



load model from local
model/model-ema1/EMA1_60To10_SPY_E150S20-Y2015-2023_ma.h5
model/model-ema1/scaler_EMA1_60To10_SPY_E150S20-Y2015-2023.gz
model/model-ema1/scaler_pred_EMA1_60To10_SPY_E150S20-Y2015-2023.gz


In [7]:
if loadModelMode=='local':
    try:
        print("Model and Scaler Object Summary")
        x_model = load_model(model_path)
    except Exception as ex:
        print(str(ex))
        raise Exception(str(ex)) 
    
    try:
        print("Scaler Max-Min")
        x_scaler = joblib.load(scale_input_path)
        x_scalerPred=joblib.load(scale_output_path)

    except Exception as ex:
        print(str(ex))
        raise Exception(str(ex))

    print("=====================================================================================================")

Model and Scaler Object Summary
Scaler Max-Min


D:\ProgramData\Anaconda3\envs\google_base\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
if loadModelMode=='gcs':
 try:    
    gcs_client = storage.Client()

    with open(scaler_file, 'wb') as scaler_f, open(scalerPred_file, 'wb') as scaler_pred_f,open(model_file, 'wb') as model_f:
        gcs_client.download_blob_to_file(scale_input_path, scaler_f
        )
        gcs_client.download_blob_to_file(scale_output_path, scaler_pred_f
        )
        gcs_client.download_blob_to_file(model_path, model_f
        )

    x_scaler = joblib.load(scaler_file)
    x_scalerPred=joblib.load(scalerPred_file)
    x_model = load_model(model_file)
 except Exception as ex:
    print(str(ex))
    raise Exception(str(ex))


In [9]:
print(x_model.summary())
#(max - min) / (X.max(axis=0) - X.min(axis=0))
print(f"max={x_scaler.data_max_} and min={x_scaler.data_min_} and scale={x_scaler.scale_}")
print(f"max={x_scalerPred.data_max_} and min={x_scalerPred.data_min_} and scale={x_scalerPred.scale_}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 180)               131040    
                                                                 
 dropout_1 (Dropout)         (None, 180)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1810      
                                                                 
Total params: 132,850
Trainable params: 132,850
Non-trainable params: 0
_________________________________________________________________
None
max=[473.99] and min=[187.09] and scale=[0.00348554]
max=[473.99] and min=[187.09] and scale=[0.00348554]


# Declare and Initialize TS Model Variable

In [10]:
date_col='Date'
prediction_col=prediction_name
feature_cols=[prediction_name]

input_sequence_length =60
output_sequence_length =10

nLastData=input_sequence_length*2

# dt_imported=datetime.now()
dt_imported=datetime.now(timezone.utc)
dtStr_imported=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
print(dtStr_imported)


2023-06-03 16:42:43


# Query Fin Data from BQ

In [15]:
lastDate=None
if today=='':
    sqlLastDate=f""" select max(Date) as LastDate  from `{table_data_id}` where Symbol='{asset_name}' """

else:
    sqlLastDate=f""" 
    select Date as LastDate  from `{table_data_id}` where Symbol='{asset_name}' 
    and Date='{today}'
    """
print(sqlLastDate)
results = client.query(sqlLastDate)
dfLastDate=results.to_dataframe()
print(dfLastDate)
if dfLastDate.empty:
    print( f"Not found price data at {today}  of {asset_name}")
    # return f"Not found price data at {today}  of {asset_name}"
else:
    lastDate=dfLastDate.iloc[0,0]
    today=lastDate.strftime('%Y-%m-%d')


print(f"Forecast {prediction_col} movement of  {asset_name} at {today}")


 
    select Date as LastDate  from `pongthorn.FinAssetForecast.fin_data` where Symbol='SPY' 
    and Date='2023-05-26'
    
     LastDate
0  2023-05-26
Forecast EMA1 movement of  SPY at 2023-05-26


In [16]:
print(f"Get last price of {asset_name}")
sqlLastPred=f"""select prediction_date,asset_name,prediction_name,pred_timestamp from `{table_id}` 
where prediction_date='{today}' and   asset_name='{asset_name}' and prediction_name='{prediction_col}'
order by pred_timestamp 
"""
print(sqlLastPred)
dfLastPred=load_data_bq(sqlLastPred)
if dfLastPred.empty==False:
   dfLastPred=dfLastPred.drop_duplicates(subset=['prediction_date','asset_name','prediction_name'],keep='last') 
   print(f"{asset_name}-{prediction_col}-{today} has been predicted price movement")
   print(dfLastPred)
   #return "there is one record of prediction price movement"
else:
   print(f"{asset_name}-{prediction_col} at {today} has not been predicted price movement yet.") 

Get last price of SPY
select prediction_date,asset_name,prediction_name,pred_timestamp from `pongthorn.FinAssetForecast.fin_movement_forecast` 
where prediction_date='2023-05-26' and   asset_name='SPY' and prediction_name='EMA1'
order by pred_timestamp 

SPY-EMA1 at 2023-05-26 has not been predicted price movement yet.


In [17]:
dayAgo=datetime.strptime(today,'%Y-%m-%d') +timedelta(days=-nLastData)
print(f"Get data from {dayAgo.strftime('%Y-%m-%d')} - {today} as input to forecast")

Get data from 2023-01-26 - 2023-05-26 as input to forecast


In [75]:
sql=f"""
SELECT  *  FROM `{table_data_id}`  
Where  {date_col} between  DATE_SUB(DATE '{today}', INTERVAL {nLastData} DAY) 
and '{today}' and Symbol='{asset_name}' order by {date_col},ImportDateTime
"""
print(sql)
query_result=client.query(sql)
df=query_result.to_dataframe()

df=df.drop_duplicates(subset=[date_col,'Symbol'],keep='last')
df[date_col]=pd.to_datetime(df[date_col],format='%Y-%m-%d')
df.set_index(date_col,inplace=True)

print(df.info())

print(df[['Symbol','Close' ,'ImportDateTime']].head())
print(df[['Symbol','Close' ,'ImportDateTime']].tail())

if df.empty==True or len(df)<input_sequence_length:
    print(f"There is enough data to make prediction during {dayAgo.strftime('%Y-%m-%d')} - {today}")
    # return "no enough data"


SELECT  *  FROM `pongthorn.FinAssetForecast.fin_data`  
Where  Date between  DATE_SUB(DATE '2023-06-01', INTERVAL 120 DAY) 
and '2023-06-01' and Symbol='SPY' order by Date,ImportDateTime

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 84 entries, 2023-02-01 to 2023-06-01
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          84 non-null     object        
 1   Close           84 non-null     float64       
 2   EMA1            84 non-null     float64       
 3   EMA2            84 non-null     float64       
 4   MACD            84 non-null     float64       
 5   SIGNAL          84 non-null     float64       
 6   ImportDateTime  84 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 5.2+ KB
None
           Symbol   Close             ImportDateTime
Date                                                
2023-02-01    SPY  410.80 2023-06-01 06

In [76]:
# plt.subplots(2, 1, figsize = (20, 10),sharex=True)

# ax1 = plt.subplot(2, 1, 1)
# plt.plot(df[['Close','EMA1','EMA2']])
# plt.ylabel('Price & EMA')

# ax2 = plt.subplot(2, 1, 2)
# plt.plot(df[['MACD','SIGNAL']])
# plt.xlabel('Date')
# plt.ylabel('MACD & Signal')

# plt.show()

# Get only Feature( 1 Indicator) to Predict itself in the next N days

In [77]:
print(f"Get Feature to Predict : {prediction_col} ")
dfForPred=df[feature_cols]
#dfForPred=dfForPred.iloc[-(input_sequence_length+1):-1,:]
dfForPred=dfForPred.iloc[-input_sequence_length:,:]
print(dfForPred.info())
print(dfForPred.shape)

print(dfForPred.head(5))
print(dfForPred.tail(5))

# dfForPred.plot(figsize = (20, 10))
# plt.show()

Get Feature to Predict : SIGNAL 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2023-03-08 to 2023-06-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SIGNAL  60 non-null     float64
dtypes: float64(1)
memory usage: 960.0 bytes
None
(60, 1)
            SIGNAL
Date              
2023-03-08 -0.1360
2023-03-09 -0.3457
2023-03-10 -0.6468
2023-03-13 -0.9973
2023-03-14 -1.2921
            SIGNAL
Date              
2023-05-25  1.3693
2023-05-26  1.3881
2023-05-30  1.4385
2023-05-31  0.6864
2023-06-01  0.8177


# Make Pediction as Forecast

In [78]:
xUnscaled=dfForPred.values #print(xUnscaled.shape)
xScaled=x_scaler.transform(xUnscaled)
print(xScaled.shape)
print(xScaled[-5:])

# # Way1
# xScaledToPredict = []
# xScaledToPredict.append(xScaled)
# print(len(xScaledToPredict))

# yPredScaled=x_model.predict(np.array(xScaledToPredict))
# print(yPredScaled.shape,yPredScaled)

# yPred  = x_scalerPred.inverse_transform(yPredScaled.reshape(-1, 1))
# print(yPred.shape,yPred)

#Way2
xScaledToPredict= xScaled.reshape(1,input_sequence_length,len(feature_cols))
print(xScaledToPredict.shape)

yPredScaled = x_model.predict(xScaledToPredict)
print(yPredScaled.shape, yPredScaled)

yPred = x_scalerPred.inverse_transform(yPredScaled).reshape(-1, 1)
print(yPred.shape, yPred)


print("============================Summary============================")
print(xUnscaled.shape)
print(yPred.shape)

# print("============================Input============================")
# print(xUnscaled)
# print("============================Output============================")
# print(yPred)



(60, 1)
[[0.76776398]
 [0.76866221]
 [0.77107023]
 [0.73513617]
 [0.74140946]]
(1, 60, 1)
1/1 [==============================] - 0s 498ms/step
(1, 10) [[0.71830523 0.6927993  0.66803455 0.64833355 0.6359732  0.6248889
  0.6156411  0.60843474 0.6040622  0.6035649 ]]
(10, 1) [[ 0.33412847]
 [-0.19971004]
 [-0.7180368 ]
 [-1.1303787 ]
 [-1.3890806 ]
 [-1.6210754 ]
 [-1.8146315 ]
 [-1.965461  ]
 [-2.0569782 ]
 [-2.0673864 ]]
============================Summary============================
(60, 1)
(10, 1)


# Build Predition Result Data

## Feature Data

In [79]:
dfFeature=pd.DataFrame(data= xUnscaled,columns=feature_cols,index=dfForPred.index)

print(dfFeature.shape)
print(dfFeature.head())
print(dfFeature.tail())

(60, 2)
            SIGNAL     Type
Date                       
2023-03-08 -0.1360  feature
2023-03-09 -0.3457  feature
2023-03-10 -0.6468  feature
2023-03-13 -0.9973  feature
2023-03-14 -1.2921  feature
            SIGNAL     Type
Date                       
2023-05-25  1.3693  feature
2023-05-26  1.3881  feature
2023-05-30  1.4385  feature
2023-05-31  0.6864  feature
2023-06-01  0.8177  feature


## Forecast Value Data

In [80]:
lastRowOfFeature=dfFeature.index.max()
firstRowofPrediction=lastRowOfFeature+timedelta(days=1)
datePred=pd.date_range(start=firstRowofPrediction,freq='b',periods=output_sequence_length)
print(datePred)

dfPrediction=pd.DataFrame(data= yPred,columns=feature_cols,index=datePred)
dfPrediction.index.name=date_col
print(dfPrediction.shape)
print(dfPrediction)


DatetimeIndex(['2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07',
               '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13',
               '2023-06-14', '2023-06-15'],
              dtype='datetime64[ns]', freq='B')
(10, 2)
              SIGNAL        Type
Date                            
2023-06-02  0.334128  prediction
2023-06-05 -0.199710  prediction
2023-06-06 -0.718037  prediction
2023-06-07 -1.130379  prediction
2023-06-08 -1.389081  prediction
2023-06-09 -1.621075  prediction
2023-06-12 -1.814631  prediction
2023-06-13 -1.965461  prediction
2023-06-14 -2.056978  prediction
2023-06-15 -2.067386  prediction


# Get Prepraed To ingest data into BQ , we have to create dataframe and convert to Json-Rowns

In [81]:
outputDF=pd.DataFrame(data=[ [today,asset_name,prediction_col,dtStr_imported] ],columns=["prediction_date","asset_name","prediction_name","pred_timestamp"])
print(outputDF.info())
print(outputDF)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prediction_date  1 non-null      object
 1   asset_name       1 non-null      object
 2   prediction_name  1 non-null      object
 3   pred_timestamp   1 non-null      object
dtypes: object(4)
memory usage: 160.0+ bytes
None
  prediction_date asset_name prediction_name       pred_timestamp
0      2023-06-01        SPY          SIGNAL  2023-06-02 06:27:45


In [82]:
jsonOutput = json.loads(outputDF.to_json(orient = 'records'))
for item in jsonOutput:
    
    dataFeature=dfFeature.reset_index()[[date_col,prediction_col]]
    dataFeature[date_col]=dataFeature[date_col].dt.strftime('%Y-%m-%d')
    dataFeature.columns=["input_date","input_feature"]
    jsonFeature= json.loads(dataFeature.to_json(orient = 'records'))
    item["feature_for_prediction"]=jsonFeature
    
    dataPred=dfPrediction.reset_index()[[date_col,prediction_col]]
    dataPred[date_col]=dataPred[date_col].dt.strftime('%Y-%m-%d')
    dataPred.columns=["output_date","output_value"]
    jsonPred= json.loads(dataPred.to_json(orient = 'records'))
    item["prediction_result"]=jsonPred
 
with open("fin_prediction.json", "w") as outfile:
    json.dump(jsonOutput, outfile)
jsonOutput

[{'prediction_date': '2023-06-01',
  'asset_name': 'SPY',
  'prediction_name': 'SIGNAL',
  'pred_timestamp': '2023-06-02 06:27:45',
  'feature_for_prediction': [{'input_date': '2023-03-08',
    'input_feature': -0.136},
   {'input_date': '2023-03-09', 'input_feature': -0.3457},
   {'input_date': '2023-03-10', 'input_feature': -0.6468},
   {'input_date': '2023-03-13', 'input_feature': -0.9973},
   {'input_date': '2023-03-14', 'input_feature': -1.2921},
   {'input_date': '2023-03-15', 'input_feature': -1.5604},
   {'input_date': '2023-03-16', 'input_feature': -1.7215},
   {'input_date': '2023-03-17', 'input_feature': -1.868},
   {'input_date': '2023-03-20', 'input_feature': -1.9535},
   {'input_date': '2023-03-21', 'input_feature': -1.9351},
   {'input_date': '2023-03-22', 'input_feature': -1.9198},
   {'input_date': '2023-03-23', 'input_feature': -1.8915},
   {'input_date': '2023-03-24', 'input_feature': -1.8243},
   {'input_date': '2023-03-27', 'input_feature': -1.7229},
   {'input_dat

# Ingest Data to BigQuery 

In [83]:
try:
    table=client.get_table(table_id)
    print("Table {} already exists.".format(table_id))
    print(table.schema)
except Exception as ex :
    print(str(ex))
#if error  please create table and other configuration as  bq_prediction.txt    

job_config = bigquery.LoadJobConfig(
# schema=[  ]
)

job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND  
#job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_json(jsonOutput,table_id, job_config = job_config)
if job.errors is not None:
    print(job.error_result)
    print(job.errors)
else:
    print(f"import to bigquery successfully  {len(jsonOutput)} records")
    
#job_config.schema

Table pongthorn.FinAssetForecast.fin_movement_forecast already exists.
[SchemaField('prediction_result', 'RECORD', 'REPEATED', None, None, (SchemaField('output_value', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('output_date', 'DATE', 'NULLABLE', None, None, (), None)), None), SchemaField('prediction_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('feature_for_prediction', 'RECORD', 'REPEATED', None, None, (SchemaField('input_feature', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('input_date', 'DATE', 'NULLABLE', None, None, (), None)), None), SchemaField('pred_timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None), SchemaField('prediction_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('asset_name', 'STRING', 'NULLABLE', None, None, (), None)]
import to bigquery successfully  1 records


In [96]:
#return   'completed job.'